In [9]:
# !pip install beautifulsoup4
# !pip install selenium

You should consider upgrading via the 'C:\Users\leedo\miniconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\leedo\miniconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from tqdm import tqdm_notebook
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import re
import urllib.request
import os
import csv

In [2]:
url = "https://www.mangoplate.com/"
driver = webdriver.Chrome("./driver/chromedriver.exe")
driver.get(url)
driver.maximize_window()

C:\Users\leedo\AppData\Local\Temp\ipykernel_15300\3248022642.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./driver/chromedriver.exe")


In [3]:
# 검색창에 서울지역 검색
search_box = driver.find_element_by_xpath('//*[@id="main-search"]')
search_box.send_keys('서울')
search_box.send_keys(Keys.RETURN)

C:\Users\leedo\AppData\Local\Temp\ipykernel_15300\615670643.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  search_box = driver.find_element_by_xpath('//*[@id="main-search"]')


In [32]:
# 세부지역 선택 강남: 1~26, 강북: 1~25
district_num = 26
# 필터 선택
driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[1]/div[1]/button[1]').click()
time.sleep(1)
# 지역 선택 서울-강남: 1, 서울-강북: 2
driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[3]/p[1]/a[{}]'.format(1)).click()

# 세부지역 선택 강남: 1~26, 강북: 1~25
driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[3]/p[2]/span[{}]/label/span'.format(district_num)).click()

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
district = soup.select("body > main > article > div.column-wrapper > div > aside > div.inner > div > div:nth-child(3) > p.metro > span:nth-child({}) > label > span".format(district_num))[0].text
district = district.replace("/", "_") + '_'

# 가격 만원미만, 1만원대, 2만원대, 3만원대 선택: 1 ~ 4
driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[2]/p/label[{}]'.format(1)).click()

# 음식종류 선택: 한식, 일식, 중식, 양식, 세계음식, 뷔페, 카페, 주점: 1~8
driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[4]/p/label[{}]'.format(1)).click()
time.sleep(1)

# 필터창 선택버튼
driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[2]/button[2]').click()
time.sleep(1)
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

C:\Users\leedo\AppData\Local\Temp\ipykernel_15300\495814539.py:4: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[1]/div[1]/button[1]').click()
C:\Users\leedo\AppData\Local\Temp\ipykernel_15300\495814539.py:7: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[3]/p[1]/a[{}]'.format(1)).click()
C:\Users\leedo\AppData\Local\Temp\ipykernel_15300\495814539.py:10: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[3]/p[2]/span[{}]/label/span'.format(district_num)).click()
C:\Users\leedo\AppData\Local\Temp\ipykernel_15300\495814539.py:18: Depreca

In [5]:
# 페이지 내 20개 식당 클릭 후 크롤링
def page_one_crawling():
    for q in range(1,11):
            for w in range(1,3):
                target = driver.find_element_by_xpath("/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[{}]/div[{}]/figure/figcaption/div/a".format(q,w))
                target.send_keys(Keys.CONTROL +"\n")
                time.sleep(0.5)
                driver.switch_to.window(driver.window_handles[-1])
                html = driver.page_source
                soup = BeautifulSoup(html, "html.parser")

                # 리뷰 개수
                review_count_out = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > header > h2 > span.RestaurantReviewList__AllCount")[0].text

                # 평점없으면 None
                try:
                    point = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > strong > span")[0].text
                except:
                    point = None


                if int(review_count_out) >= 1:

                    # 더보기 칸까지 스크롤 내리기
                    try:
                        driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]')
                        action = ActionChains(driver)
                        action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]')).perform()
                    except:
                        pass
                    try:
                        for e in range(int(review_count_out)//5):
                            driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]').click()
                            time.sleep(0.8)

                            action = ActionChains(driver)
                            action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]')).perform()
                            time.sleep(0.1)
                    except:
                        pass
                    time.sleep(0.8)
                    html = driver.page_source
                    soup = BeautifulSoup(html, "html.parser")

                    # 스크롤 내려서 삭제된 리뷰 제외된 실제 개수
                    review_count = len(soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul")[0]) -1

                    # 실제 리뷰개수가 1개부터 조사
                    if review_count >= 1:
                        # 가게 이름
                        name = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > h1")[0].text
                        save_path = "/Users/leedo/Documents/ds_study/" + district + name + "/"

                        # 지역 + 가게 이름으로 폴더만들기
    #                     try:
    #                         if not os.path.isdir(save_path):
    #                             os.mkdir(save_path)
    #                     except:
    #                         pass

                        detail_count = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr > th")

                        for i in range(len(detail_count)):
                            if detail_count[i].text == '주소':
                                address = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child({})".format(i + 1))[0].text

                                address = re.sub('\n', ' ', address).strip().split(' ')

                                for i in range(len(address)):
                                    if address[i] == '지번':
                                        a=address[1:i]
                                address = ' '.join(a)


                            elif detail_count[i].text == '음식 종류':

                                category = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child({})".format(i + 1))[0].text
                                category = re.sub('\n', ' ', category).strip().split(' ')
                                category = category[3:]
                                category = ' '.join(category)

                            elif detail_count[i].text == '가격대':

                                price = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child({})".format(i + 1))[0].text
                                price = re.sub('\n', ' ', price).strip().split(' ')
                                price = price[1:]
                                price = ' '.join(price)

                        time.sleep(0.1)
                        
                        # 스크롤 내려서 삭제된 리뷰 제외된 실제 개수
                        review_count = int(len(soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul")[0]) -1)
                                                
                        # 리뷰 크롤링
                        try:
                            for i in range(1,int(review_count)+1):
                                tag = "body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul > li:nth-child({}) > a > div.RestaurantReviewItem__ReviewContent > div > p".format(i)
                                review = soup.select(tag)
                                review = review[0].text
                                review = str(re.sub('\n', ' ', review).strip())
                                review_id = soup.select('body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul > li:nth-child({}) > a > div.RestaurantReviewItem__User > span'.format(i))[0].text
                                
                                # 맛있다 또는 괜찮다 표시
                                recommend = driver.find_element_by_xpath("/html/body/main/article/div[1]/div[1]/div/section[3]/ul/li[{}]/a/div[3]/span".format(i)).text
                                if review != '' and review != '.':
                                    review_list.append({'가게이름':name,'리뷰아이디':review_id, '리뷰':review, '맛평가': recommend})
                                elif review == '' or review == '.':
                                    review_count -= 1
                        except:
                            pass

                        detail.append({'가게이름':name,'가게 주소':address,'음식 종류':category,'가격 범위':price,"리뷰개수":review_count,'평점':point})

                        # 리뷰 펼친 후 스크롤이 해당 지점에 못 미쳐서 클릭 조회가 안되던 걸 리뷰 개수 위치로 스크롤 이동 후 해결
                        action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/header/h2/span[4]')).perform()
                        # n 번째 리뷰 클릭후 썸네일 다운
                        count = 0

                        try:               
    #                         for r in range(review_count):
    #                             tag = '/html/body/main/article/div[1]/div[1]/div/section[3]/ul/li[{}]/a/div[2]/div/p'.format(r+1)
    #                             driver.find_element_by_xpath(tag).click()

    #                             action = ActionChains(driver)
    #                             action.move_to_element(driver.find_element_by_xpath(tag)).perform()

    #                             time.sleep(0.5)

    #                             driver.switch_to.window(driver.window_handles[-1])
    #                             html = driver.page_source
    #                             soup = BeautifulSoup(html, "html.parser")

    #                             # 썸네일 갯수
    #                             html = driver.page_source
    #                             soup = BeautifulSoup(html, "html.parser")
    #                             if len(soup.select("body > main > section > div > div")) != 0:
    #                                 picture_count = int((len(soup.select("body > main > section > div > div")[0]) - 1) / 2)

    #                                 # n 번째 썸네일 url확인 후 저장
    #                                 for t in range(picture_count):
    #                                     count += 1
    #                                     try:
    #                                         image_url = driver.find_element_by_xpath("/html/body/main/section/div/div/div[{}]".format(t+1)).get_attribute("style")
    #                                         a = image_url.index('(')
    #                                         b = image_url.index(')')
    #                                         image_url = image_url[a+2:b-1]
    #                                         urllib.request.urlretrieve(image_url, save_path +'{}.jpg'.format(count))
    #                                         time.sleep(0.1)
    #                                     except:
    #                                         pass
    #                                 driver.close()  #링크 이동 후 탭 닫기
    #                                 driver.switch_to.window(driver.window_handles[-1])
    #                                 time.sleep(0.1)
    #                             else:
    #                                 driver.close()  #링크 이동 후 탭 닫기
    #                                 driver.switch_to.window(driver.window_handles[-1])
    #                                 time.sleep(0.1)

                            driver.close()  #링크 이동 후 탭 닫기
                            driver.switch_to.window(driver.window_handles[-1])
                            time.sleep(0.3)
                        except:
                            driver.close()  #링크 이동 후 탭 닫기
                            driver.switch_to.window(driver.window_handles[-1])
                            time.sleep(0.3)

                    else:            
                        driver.close()  #링크 이동 후 탭 닫기
                        driver.switch_to.window(driver.window_handles[-1])
                        time.sleep(0.3)
                else:
                    driver.close()  #링크 이동 후 탭 닫기
                    driver.switch_to.window(driver.window_handles[-1])
                    time.sleep(0.3)

In [6]:
def page_start_to_end_crawling():
    for z in range(2,11):
        try:
            page_one_crawling()
            driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[4]/p/a[{}]'.format(z)).click()
            time.sleep(1)
            if z == 10:
                page_one_crawling()
                time.sleep(1)
        except:
            break

In [7]:
def filter_price_button_option():
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # 필터선택 버튼 스크롤
    action = ActionChains(driver)
    action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[1]/div[1]/button[1]')).perform()
    time.sleep(0.3)
    
    # 필터 선택
    driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[1]/div[1]/button[1]').click()
    time.sleep(0.3)

    # 가격선택 버튼 스크롤
    action = ActionChains(driver)
    action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[2]/p/label[1]')).perform()
    time.sleep(0.3)

    # 가격 만원미만, 1만원대, 2만원대, 3만원대 선택: 1 ~ 4
    driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[2]/p/label[{}]'.format(x)).click()
    driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[2]/p/label[{}]'.format(x+1)).click()
    time.sleep(0.3)

    # 필터창 선택버튼
    driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[2]/button[2]').click()
    time.sleep(0.3)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

In [8]:
def filter_category_button_option():
    # 필터선택 버튼 스크롤
    action = ActionChains(driver)
    action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[1]/div[1]/button[1]')).perform()    
    time.sleep(0.3)
    
    # 필터 선택
    driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[1]/div[1]/button[1]').click()
    time.sleep(0.3)

    driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[2]/p/label[1]').click()
    driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[2]/p/label[{}]'.format(x+1)).click()

    # 음식종류선택 버튼 스크롤
    action = ActionChains(driver)
    action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[4]/p/label[5]')).perform()
    time.sleep(0.3)
    
    # 음식종류 선택: 한식, 일식, 중식, 양식, 세계음식, 뷔페, 카페, 주점: 1~8
    driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[4]/p/label[{}]'.format(s)).click()
    driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[4]/p/label[{}]'.format(s+1)).click()
    time.sleep(0.3)

    # 필터창 선택버튼
    driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[2]/button[2]').click()
    time.sleep(0.3)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

In [33]:
detail = []
review_list = []

In [34]:
# total
# 리뷰만 크롤링
# 1 - 10 페이지, 2부터 시작
for s in range(1,9):
    try:
        for x in range(1,4):
            page_start_to_end_crawling()
            filter_price_button_option()
            
            if x == 3:
                page_start_to_end_crawling()
        filter_category_button_option()
    except:
        pass

C:\Users\leedo\AppData\Local\Temp\ipykernel_15300\441350944.py:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  target = driver.find_element_by_xpath("/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[{}]/div[{}]/figure/figcaption/div/a".format(q,w))
C:\Users\leedo\AppData\Local\Temp\ipykernel_15300\441350944.py:26: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]')
C:\Users\leedo\AppData\Local\Temp\ipykernel_15300\441350944.py:28: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]')).perform()
C:\Users\leedo\AppData\Local\Temp\ipykernel_15300\441350944.py:33: DeprecationWarn

In [35]:
review_frame = pd.DataFrame(review_list)
review_frame.index = review_frame.index + 1
review_frame

,가게이름,리뷰아이디,리뷰,맛평가
1,청담91떡볶이,정은,쫄볶이 / 91김밥 / 오뎅,맛있다
2,청담91떡볶이,도니,"우연히 주변에 왔다가 빠르게 먹고가야하는데 떡볶이가 땡겨서 들러본 곳!쫄볶이, 91...",맛있다
3,청담91떡볶이,제이,** 청담동 떡볶이는 여기질좋은 고추가루 맛의 떡볶이를 만날 수 있는 곳이예요.위치...,맛있다
4,청담91떡볶이,Hailey,상호명이 그냥 분식집이 아니라 떡볶이인것 부터 느낄 수 있듯이 은근 숨은 떡볶이 맛...,맛있다
5,청담91떡볶이,YNZ,스팸김밥과 떡볶이가 맛있는 곳! 가게가 워낙 조그맣고 좁아서 포장해다 먹었다. 스팸...,괜찮다
...,...,...,...,...
4534,레 끌레 드 크리스탈,미식한고독가,와인샵으로 알고 있는 사람들이 많은데 샵도 있고 음식까지 나오는 곳은 아마 여기 하...,맛있다
4535,레 끌레 드 크리스탈,지나가는 강아지,냠냠자주갈것같다,맛있다
4536,더부즈,라물장,"원래는 30세 이상을 위한 곳이지만, 지인 덕분에 한번 방문해 볼 수 있었습니다. ...",맛있다
4537,클래시크,팩폭윰,와인마시면서 재즈 듣기 좋아요!공연피가 없어서 그런지 메뉴금액들은 다소 높은편,맛있다


In [36]:
detail_frame = pd.DataFrame(detail)
detail_frame.index = detail_frame.index + 1
detail_frame

,가게이름,가게 주소,음식 종류,가격 범위,리뷰개수,평점
1,청담91떡볶이,서울특별시 강남구 도산대로57길 8,기타 한식,만원 미만,6,4.2
2,남산터,서울특별시 강남구 선릉로152길 10 1F,탕 / 찌개 / 전골,만원 미만,14,4.1
3,마녀김밥,서울특별시 강남구 압구정로79길 32 1F,기타 한식,만원 미만,30,4.0
4,두루,서울특별시 강남구 선릉로148길 28,국수 / 면 요리,만원 미만,7,4.0
5,장독대김치찌개,서울특별시 강남구 도산대로102길 5,탕 / 찌개 / 전골,만원 미만,12,3.6
...,...,...,...,...,...,...
457,마토,서울특별시 강남구 선릉로146길 23 4F,칵테일 / 와인,4만원 이상,1,None
458,레 끌레 드 크리스탈,서울특별시 강남구 압구정로79길 8 하나빌딩 2-3F,칵테일 / 와인,4만원 이상,2,None
459,더부즈,서울특별시 강남구 도산대로85길 13 B1,칵테일 / 와인,3만원-4만원,1,None
460,클래시크,서울특별시 강남구 압구정로79길 81 B1,칵테일 / 와인,4만원 이상,1,None


In [37]:
district

'청담동_'

In [14]:
import googlemaps
from tqdm import tqdm

In [15]:
gmaps_key = ''
gmaps = googlemaps.Client(key=gmaps_key)

In [36]:
lat = []
lng = []
for i in tqdm(detail_frame['가게 주소']):
    lat.append(gmaps.geocode('{}'.format(i), language='ko')[0].get('geometry')['location']['lat'])
    lng.append(gmaps.geocode('{}'.format(i), language='ko')[0].get('geometry')['location']['lng'])

100%|████████████████████████████████████████████████████████████████████████████████| 390/390 [01:39<00:00,  3.94it/s]


In [37]:
detail_frame['lat'] = lat
detail_frame['lng'] = lng
detail_frame

,가게이름,가게 주소,음식 종류,가격 범위,리뷰개수,평점,lat,lng
1,방이샤브샤브칼국수,서울특별시 송파구 오금로31길 42,국수 / 면 요리,만원 미만,30,4.3,37.509248,127.123337
2,황산냉면,서울특별시 송파구 가락로 257,기타 한식,만원 미만,12,4.1,37.510909,127.118439
3,샛집남원추어탕,서울특별시 송파구 오금로11길 7-12,탕 / 찌개 / 전골,만원 미만,7,4.1,37.514824,127.107732
4,방이손칼국수,서울특별시 송파구 백제고분로48길 10,국수 / 면 요리,만원 미만,4,3.8,37.511323,127.114250
5,백암왕순대,서울특별시 송파구 위례성대로18길 27-6 삼영빌딩,탕 / 찌개 / 전골,만원 미만,15,3.6,37.509604,127.125388
...,...,...,...,...,...,...,...,...
386,호호부엌,서울특별시 송파구 오금로19길 16 1F,칵테일 / 와인,2만원-3만원,3,None,37.511043,127.114105
387,코코라운지,서울특별시 송파구 올림픽로32길 7 호텔더캐슬 방이점 17F,칵테일 / 와인,2만원-3만원,2,None,37.515418,127.109082
388,오고보,서울특별시 송파구 올림픽로34길 27-23 2F,칵테일 / 와인,2만원-3만원,2,None,37.515094,127.112028
389,노즈,서울특별시 송파구 오금로17길 4-7 101호,칵테일 / 와인,2만원-3만원,1,None,37.511086,127.111798


In [38]:
review_frame.to_csv('C:/Users/leedo/Documents/ds_study/{}_review_list.csv'.format(district + 'total'), encoding='utf-8-sig')

In [39]:
detail_frame.to_csv('C:/Users/leedo/Documents/ds_study/{}_detail.csv'.format(district + 'total'), encoding='utf-8-sig')